In [1]:
# !pip install langchain_openai langchain_community langchain pymysql chromadb -q

In [2]:
# !pip install --upgrade --quiet langchain-openai

In [1]:
import os
from langchain_community.utilities.sql_database import SQLDatabase
from dotenv import load_dotenv
from langchain.chains.sql_database.query import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI,AzureChatOpenAI,OpenAIEmbeddings


In [2]:
load_dotenv()

True

In [21]:

# os.environ["O_OPENAI_API_KEY"] = ""

db_user = "root"
db_password = "pass"
db_host = "localhost"
db_name = "demo_db"

# db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)


mysql
['pages', 'permission', 'roles', 'tblrteventrequestdata', 'tblrteventrequeststatus']

CREATE TABLE pages (
	`PageId` INTEGER NOT NULL AUTO_INCREMENT, 
	`PageName` VARCHAR(50) NOT NULL, 
	PRIMARY KEY (`PageId`)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from pages table:
PageId	PageName
1	Others
2	Geo Master
3	Stores
*/


CREATE TABLE permission (
	`PermissionId` INTEGER NOT NULL AUTO_INCREMENT, 
	`RoleID` INTEGER, 
	`PageID` INTEGER, 
	`CanView` BIT(1) NOT NULL, 
	`CanEdit` BIT(1) NOT NULL, 
	PRIMARY KEY (`PermissionId`)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from permission table:
PermissionId	RoleID	PageID	CanView	CanEdit
1	1	1	0	0
2	1	2	0	0
3	1	3	0	0
*/


CREATE TABLE roles (
	`RoleId` INTEGER NOT NULL AUTO_INCREMENT, 
	`RoleName` VARCHAR(50) NOT NULL, 
	PRIMARY KEY (`RoleId`)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from roles table:
RoleId	RoleName
1	Super Admin
2	Admi

In [22]:
OPENAI_API_KEY = os.getenv("O_OPENAI_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

In [26]:
llm_openai = ChatOpenAI(openai_api_key=OPENAI_API_KEY,model="gpt-3.5-turbo", temperature=0)
generate_query = create_sql_query_chain(llm_openai, db)


In [6]:
query = generate_query.invoke({"question": "what is price of `1968 Ford Mustang`"})
# "what is price of `1968 Ford Mustang`"
print(query)

SELECT `buyPrice`, `MSRP` 
FROM products 
WHERE `productName` = '1968 Ford Mustang'
LIMIT 1;


In [27]:
query = generate_query.invoke({"question": "Retrieve the roles that have permissions on pages with IDs divisible by 3."})
# "what is price of `1968 Ford Mustang`"
print(query)

SELECT DISTINCT r.RoleName
FROM roles r
JOIN permission p ON r.RoleId = p.RoleID
WHERE p.PageID % 3 = 0;
LIMIT 5


In [23]:
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
azure_api_version=os.getenv("AZURE_OPENAI_API_VERSION1"),
azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE1")

# openai.api_type = "azure"
# openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.api_base = os.getenv("OPENAI_API_BASE") # Endpoint
# openai.api_version = os.getenv("OPENAI_API_VERSION")
# openai.deployment_name = os.getenv("DEPLOYMENT_NAME")
# # openai_deployment = "chat-gpt35"
openai_deployment = os.getenv("AZURE_DEPLOYMENT_NAME1")
# openai_model = "gpt-35-turbo"

In [24]:
llm_azureopenai = AzureChatOpenAI(openai_api_key=AZURE_OPENAI_API_KEY, openai_api_version = '2023-07-01-preview',azure_endpoint="https://qaazureaiservice.openai.azure.com/", azure_deployment=openai_deployment)

generate_query_az = create_sql_query_chain(llm_azureopenai, db)


In [9]:
query_az = generate_query_az.invoke({"question": "what is price of `1968 Ford Mustang`"})

print(query_az)

SELECT `buyPrice` FROM products WHERE `productName` = '1968 Ford Mustang' LIMIT 1;


In [25]:
query_az = generate_query_az.invoke({"question": "Retrieve the roles that have permissions on pages with IDs divisible by 3."})
print(query_az)

SELECT roles.RoleName FROM roles
JOIN permission ON roles.RoleId = permission.RoleID
WHERE permission.PageID % 3 = 0
LIMIT 5;


In [28]:
query_az = generate_query_az.invoke({"question": "Retrieve a list of roles and the number of pages each role has permission to view."})
print(query_az)

SELECT r.RoleName, COUNT(p.PageID) AS NumOfPages 
FROM roles r 
LEFT JOIN permission p ON r.RoleId = p.RoleID AND p.CanView = 1 
GROUP BY r.RoleId 
ORDER BY NumOfPages DESC 
LIMIT 5;


In [35]:
query_az = generate_query_az.invoke({"question": "Find the pages with the highest number of roles having view access."})
print(query_az)

SELECT pages.PageName, COUNT(permission.RoleID) AS NumRoles
FROM pages
JOIN permission ON pages.PageId = permission.PageID
WHERE permission.CanView = 1
GROUP BY pages.PageId
ORDER BY NumRoles DESC
LIMIT 5;


In [30]:
query_az = generate_query_az.invoke({"question": "Count the number of events for each year."})
print(query_az)

SELECT YEAR(EventDate) AS `Year`, COUNT(*) AS `Number of Events` FROM tblrteventrequestdata GROUP BY `Year`


In [32]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [33]:
execute_query = QuerySQLDataBaseTool(db=db)

In [12]:
execute_query.invoke(query)

"[(Decimal('95.34'), Decimal('194.57'))]"

In [13]:
execute_query.invoke(query_az)

"[(Decimal('95.34'),)]"

In [36]:
execute_query.invoke(query_az)

"[('View Resource Materials', 6), ('Agent Assignment', 4), ('Approve Event', 4), ('Assign Store', 4), ('Users', 3)]"

In [14]:
chain = generate_query | execute_query
chain.invoke({"question": "How many orders are there"})

'[(326,)]'

In [15]:
chain = generate_query_az | execute_query
chain.invoke({"question": "How many orders are there"})

'[(326,)]'

In [16]:
chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

In [17]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

rephrase_answer = answer_prompt | llm_azureopenai | StrOutputParser()

chain = (
        RunnablePassthrough.assign(query=generate_query_az).assign(
            result=itemgetter("query") | execute_query
        )
        | rephrase_answer
)

chain.invoke({"question": "How many orders are there"})

'There are 326 orders.'

Adding few-shot examples

In [18]:
examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT MAX(amount) FROM payments;"
    },
    {
        "input": "Show product details for products in the 'Motorcycles' product line.",
        "query": "SELECT * FROM products WHERE productLine = 'Motorcycles';"
    },
    {
        "input": "Retrieve the names of employees who report to employee number 1002.",
        "query": "SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;"
    },
    {
        "input": "List all products with a stock quantity less than 7000.",
        "query": "SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;"
    },
    {
        'input':"what is price of `1968 Ford Mustang`",
        "query": "SELECT `buyPrice`, `MSRP` FROM products  WHERE `productName` = '1968 Ford Mustang' LIMIT 1;"
    }
]

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # input_variables=["input","top_k"],
    input_variables=["input1"],
)
print(few_shot_prompt.format(input1="How many products are there?",top_k=3,table_info=""))

Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: Retrieve the names of employees who report to employee number 1002.
SQLQuery:
AI: SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: what is price of `1968 Ford Mustang`
SQLQuery:
AI: SELECT `buyPrice`, `MSRP` FROM products  WHERE `productName` = '1968 Ford Mustang' LIMIT 1;


Dynamic few-shot example selection

In [20]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
# from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    vectorstore,
    k=2,
    input_keys=["input"],
)
example_selector.select_examples({"input": "how many employees we have?"})
# example_selector.select_examples({"input": "How many employees?"})

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"],
)
print(few_shot_prompt.format(input="How many products are there?"))
     